In [1]:
from exp.nb_01 import *

In [2]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [3]:
x_train,y_train,x_valid,y_valid = get_data(mnist_path)
train_ds,valid_ds = Dataset(x_train, y_train),Dataset(x_valid, y_valid)
nh,bs = 50,64
c = y_train.max().item()+1
loss_func = F.cross_entropy

In [4]:
class DataBunch():
    def __init__(self, train_dl, valid_dl, c=None):
        self.train_dl, self.valid_dl, self.c = train_dl, valid_dl, c
        
    @property
    def train_ds(self): return self.train_dl.dataset
    
    @property
    def valid_ds(self): return self.valid_dl.dataset

In [5]:
data = DataBunch(*get_dls(train_ds, valid_ds, bs), c)

In [10]:
#export
from torch import optim

In [8]:
#export
def get_model(data, lr=0.5, nh=50):
    model=nn.Sequential(nn.Linear(data.train_ds.x.shape[1],nh), nn.ReLU(), nn.Linear(nh, 10))
    return model, optim.SGD(model.parameters(), lr=lr)

class Learner():
    def __init__(self, model, opt, loss_func, data):
        self.model, self.opt, self.loss_func, self.data=model, opt, loss_func, data

In [11]:
learn = Learner(*get_model(data), loss_func, data)

In [12]:
def fit(epochs, learn):
    for epoch in range(epochs):
        learn.model.train()
        for xb,yb in learn.data.train_dl:
            loss = learn.loss_func(learn.model(xb), yb)
            loss.backward()
            learn.opt.step()
            learn.opt.zero_grad()

        learn.model.eval()
        with torch.no_grad():
            tot_loss,tot_acc = 0.,0.
            for xb,yb in learn.data.valid_dl:
                pred = learn.model(xb)
                tot_loss += learn.loss_func(pred, yb)
                tot_acc  += accuracy (pred,yb)
        nv = len(learn.data.valid_dl)
        print(epoch, tot_loss/nv, tot_acc/nv)
    return tot_loss/nv, tot_acc/nv

In [13]:
loss,acc = fit(1, learn)

0 tensor(0.2754) tensor(0.9158)
